# Earthquake Forecasting with RF

## Setup Enviroment 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
import re
import optuna

In [65]:
from sklearn.ensemble import HistGradientBoostingRegressor
import joblib

In [2]:
pd.set_option("display.max_rows",1000)

In [12]:
df2_ = pd.read_excel("kaggle_test2.xlsx",index_col="Unnamed: 0")
df_ = pd.read_csv("dataset.csv", encoding="unicode-escape", sep=",",on_bad_lines="skip",index_col="Unnamed: 0")

In [13]:
df = df_.copy()

## EDA on Main Dataset

In [14]:
# Değişken isimlerini küçültüyorum
df.columns = [col.lower() for col in df.columns]

# Bağımlı değişkenin türevlerini overfit olmaması için çıkartıyorum.
df.drop(["md","ml","mw","ms","mb"],axis=1,inplace=True)

# Date değişkeninin oluşturulması
df["olus tarihi"] = df["olus tarihi"].astype(str)
df["olus zamani"] = df["olus zamani"].astype(str)
# tip değişkeni 2 sınıf ve SM sınıfından sadece 2 gözlem var. O yüzden dropluyorum bu kolonu.
df["depremin_olus_zamani"] = df["olus tarihi"] + " " + df["olus zamani"]
df.drop(["olus tarihi", "olus zamani", "tip"], axis = 1, inplace = True)
df["depremin_olus_zamani"] = pd.to_datetime(df["depremin_olus_zamani"])

# Çoğullanmış ID'leri düşürüyorum.
df.drop_duplicates(subset=["deprem kodu"],inplace = True)

# Türkçe karakterleri İngilizce karakterlere dönüştüren bir çeviri sözlüğü oluşturalım
turkce_ingilizce_ceviri = {
    'ç': 'c',
    'ğ': 'g',
    'ı': 'i',
    'ö': 'o',
    'ş': 's',
    'ü': 'u',
    'Ç': 'C',
    'Ğ': 'G',
    'İ': 'I',
    'Ö': 'O',
    'Ş': 'S',
    'Ü': 'U'
}
df["yer"] = df["yer"].apply(lambda yer: yer.translate(str.maketrans(turkce_ingilizce_ceviri)))

# Şehir isimlerini çekmek için düzenli ifade kullanalım

def extract_cities(dataframe,column):
  sadece_sehirler = []
  for yer in dataframe[column]:
      eslesme = re.search(r'\((.*?)\)', yer)  # Parantez içindekileri kontrol et
      if eslesme:
          sadece_sehirler.append(eslesme.group(1))  # Parantez içindeki değeri ekle
      else:
          sadece_sehirler.append(yer)  # Parantez içinde değer yoksa doğrudan ekle
  return sadece_sehirler

df["temizlenmis_yer"] = extract_cities(df,"yer")

# Köşeli parantezleri atıyoruz. Böylelikle o şehir/alandaki depremlere odaklanmış oluyoruz.
df["temizlenmis_yer"] = df["temizlenmis_yer"].apply(lambda yer: re.sub(r'\[.*?\]', '', yer))

# Yerin neresi olduğu belli olmadığı için çıkartıyorum.
df = df[~(df["temizlenmis_yer"] == "#NAME?")]

In [15]:
df.temizlenmis_yer.nunique()

368

## Joining Two Tables

In [16]:
df = pd.concat([df,df2_],axis=0,ignore_index=True)

In [17]:
df.head()

,no,deprem kodu,enlem,boylam,derinlik,xm,yer,depremin_olus_zamani,temizlenmis_yer
0,1.0,2.023043e+13,38.3392,37.7633,8.7,4.3,KEPEZ-AKCADAG (MALATYA) [East 1.7 km],2023-04-30 13:01:27.690,MALATYA
1,2.0,2.023043e+13,40.8077,31.0708,3.7,3.5,PASAKONAGI- (DUZCE) [South West 0.9 km],2023-04-30 04:02:53.490,DUZCE
2,3.0,2.023043e+13,35.6422,34.0252,22.4,3.5,AKDENIZ,2023-04-30 02:51:22.150,AKDENIZ
3,4.0,2.023043e+13,37.8623,36.2185,5.0,4.0,KARAKUYU-SAIMBEYLI (ADANA) [East 1.4 km],2023-04-29 19:48:32.870,ADANA
4,5.0,2.023043e+13,38.0220,36.4457,5.0,3.7,YIRICEK-GOKSUN (KAHRAMANMARAS) [North West 2....,2023-04-29 17:58:29.200,KAHRAMANMARAS


In [18]:
df.temizlenmis_yer.nunique()

368

## Feature Engineering

### Creating DateTime

In [19]:
def creating_datetime_feature(dataframe):
    dataframe["ay"] = dataframe["depremin_olus_zamani"].dt.month
    dataframe["yil"] = dataframe["depremin_olus_zamani"].dt.year
    dataframe["ay_gun"] = dataframe["depremin_olus_zamani"].dt.day
    dataframe["hafta_gun"] = dataframe["depremin_olus_zamani"].dt.dayofweek
    dataframe["saat"] =  dataframe["depremin_olus_zamani"].dt.hour
    dataframe["aksam"] = ((dataframe['saat'] >= 18) & (dataframe['saat'] <= 23)) | ((dataframe['saat'] >= 0) & (dataframe['saat'] <= 4))
    dataframe['yıl_gün'] = dataframe["depremin_olus_zamani"].dt.dayofyear
    dataframe['yıl_hafta'] = dataframe["depremin_olus_zamani"].dt.weekofyear
    dataframe["ay_basi"] = dataframe["depremin_olus_zamani"].dt.is_month_start.astype(int)
    dataframe["ay_sonu"] = dataframe["depremin_olus_zamani"].dt.is_month_end.astype(int)
    dataframe["çeyreklik"] = dataframe["depremin_olus_zamani"].dt.quarter
    return dataframe
df = creating_datetime_feature(df)

C:\Users\oftomurcuk\AppData\Local\Temp\ipykernel_23528\4216593417.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  dataframe['yıl_hafta'] = dataframe["depremin_olus_zamani"].dt.weekofyear


In [20]:
df.head()

,no,deprem kodu,enlem,boylam,derinlik,xm,yer,depremin_olus_zamani,temizlenmis_yer,ay,yil,ay_gun,hafta_gun,saat,aksam,yıl_gün,yıl_hafta,ay_basi,ay_sonu,çeyreklik
0,1.0,2.023043e+13,38.3392,37.7633,8.7,4.3,KEPEZ-AKCADAG (MALATYA) [East 1.7 km],2023-04-30 13:01:27.690,MALATYA,4,2023,30,6,13,False,120,17,0,1,2
1,2.0,2.023043e+13,40.8077,31.0708,3.7,3.5,PASAKONAGI- (DUZCE) [South West 0.9 km],2023-04-30 04:02:53.490,DUZCE,4,2023,30,6,4,True,120,17,0,1,2
2,3.0,2.023043e+13,35.6422,34.0252,22.4,3.5,AKDENIZ,2023-04-30 02:51:22.150,AKDENIZ,4,2023,30,6,2,True,120,17,0,1,2
3,4.0,2.023043e+13,37.8623,36.2185,5.0,4.0,KARAKUYU-SAIMBEYLI (ADANA) [East 1.4 km],2023-04-29 19:48:32.870,ADANA,4,2023,29,5,19,True,119,17,0,0,2
4,5.0,2.023043e+13,38.0220,36.4457,5.0,3.7,YIRICEK-GOKSUN (KAHRAMANMARAS) [North West 2....,2023-04-29 17:58:29.200,KAHRAMANMARAS,4,2023,29,5,17,False,119,17,0,0,2


NaN gelen değerler, iki tablonun joinlenmesinden dolayı oluşuyor. İleride türeteceğimiz Lag featurlar'da bunalara benzer mantıkta NaN çıkan olacak.

Joinlemeden önceki saatleri göster.

In [21]:
# Akşam değişkeni Integer tipine dönüştürülüyor. 
df["aksam"] = df['aksam'].astype(int)

### Lag Feature

In [22]:
# random noise. Bağımlı değişkenden türeteceğimiz featurelarda, overfitin önüne geçmek için gürültü oluşturacağız.
def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe)))

In [23]:
# Zamana göre sıralıyoruz Zaman Serilerini.
df = df.sort_values(by = ["depremin_olus_zamani","temizlenmis_yer"],
               axis = 0)

In [24]:
# lag features. Shift ile gecikmeyi sağlıyoruz. 
# Random noise ile gürültü ekliyoruz. 
# Transform ile de zaten dönüştürme yapıyoruz.
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['deprem_lag_' + str(lag)] = dataframe.groupby(["temizlenmis_yer"])['xm'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe

In [25]:
# çeşitli shiftler/lagler/gecikmeler girelim. Gecikme  o zamanki değer demekti unutma. 3 ay ve katları olacak şekilde bakıyorum! Quarter aldım.
# Gelecekten Not: Şuanlık veri setinde gün gün gözlem var. Bunları çeyreklik yapınca SMAPE değerimiz arttı!
df = lag_features(df, [1, 3, 7, 30, 60, 90, 180, 360])

In [26]:
df.isnull().sum()

no                       2208
deprem kodu              2208
enlem                       0
boylam                      0
derinlik                    0
xm                       2208
yer                      2208
depremin_olus_zamani        0
temizlenmis_yer             0
ay                          0
yil                         0
ay_gun                      0
hafta_gun                   0
saat                        0
aksam                       0
yıl_gün                     0
yıl_hafta                   0
ay_basi                     0
ay_sonu                     0
çeyreklik                   0
deprem_lag_1             2208
deprem_lag_3             2208
deprem_lag_7             2576
deprem_lag_30            4999
deprem_lag_60            6880
deprem_lag_90            8363
deprem_lag_180          11382
deprem_lag_360          14531
dtype: int64

Lag verdiğimiz için eksik değer çıkmasını bekliyoruz ve normal karşılıyoruz. Ağaç yöntemleri kullanacağımız için eksik verileri handle etmek için ek çaba sarfetmeyeceğiz. Varyansı bozmak istemiyoruz.

### Rolling Mean Feature

In [27]:
# Rolling Mean Feature
# Window parametresi, adım sayısını belirtmek için kullanılıyor.
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['deprem_roll_mean_' + str(window)] = dataframe.groupby(["temizlenmis_yer"])['xm']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(
            dataframe)
    return dataframe

In [28]:
# Min Periodu 10 yaptığımız için min window'a da 10 girebiliyoruz.
df = roll_mean_features(df, [10, 30, 60, 90, 180, 360])

In [29]:
df.isnull().sum()

no                       2208
deprem kodu              2208
enlem                       0
boylam                      0
derinlik                    0
xm                       2208
yer                      2208
depremin_olus_zamani        0
temizlenmis_yer             0
ay                          0
yil                         0
ay_gun                      0
hafta_gun                   0
saat                        0
aksam                       0
yıl_gün                     0
yıl_hafta                   0
ay_basi                     0
ay_sonu                     0
çeyreklik                   0
deprem_lag_1             2208
deprem_lag_3             2208
deprem_lag_7             2576
deprem_lag_30            4999
deprem_lag_60            6880
deprem_lag_90            8363
deprem_lag_180          11382
deprem_lag_360          14531
deprem_roll_mean_10      3741
deprem_roll_mean_30      3226
deprem_roll_mean_60      3226
deprem_roll_mean_90      3226
deprem_roll_mean_180     3226
deprem_rol

### Exponentially Weighted Mean Feature

In [30]:
# Shift ile gecikme veriyor
# ewm ile üs alıyor,
# en sonrada ortalamasını alıp transform ile dönüştürüyoruz.
def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['deprem_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["temizlenmis_yer"])['xm'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

In [31]:
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [1, 3, 7, 30, 60, 90, 180, 360]

In [32]:
df = ewm_features(df, alphas, lags)

In [33]:
df.isnull().sum()

no                               2208
deprem kodu                      2208
enlem                               0
boylam                              0
derinlik                            0
xm                               2208
yer                              2208
depremin_olus_zamani                0
temizlenmis_yer                     0
ay                                  0
yil                                 0
ay_gun                              0
hafta_gun                           0
saat                                0
aksam                               0
yıl_gün                             0
yıl_hafta                           0
ay_basi                             0
ay_sonu                             0
çeyreklik                           0
deprem_lag_1                     2208
deprem_lag_3                     2208
deprem_lag_7                     2576
deprem_lag_30                    4999
deprem_lag_60                    6880
deprem_lag_90                    8363
deprem_lag_1

### One Hot Encoding for temislenmis_yer

In [34]:
df = pd.get_dummies(df, columns=['temizlenmis_yer'])

In [35]:
df.shape

(22111, 441)

**Gerçekleşen depremlerin sadece %5'i 5.0'dan büyük.**

### Bağımlı Değişkene Log1p Dönüşümünün Uygulanması

Bunu yapmamızın sebebi, zamandan tasarruf sağlamak.

In [36]:
df['xm'] = np.log1p(df["xm"].values)

## Custom Cost Func For HistGradientBoostingregressor

RFRegressor NaN value kabul etmediği için bu HGBR'a dönmek zorunda kaldık.

In [54]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val

def hist_gradient_boosting_smape(preds, train_data):
    labels = train_data
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False

## Time-Based Validation Sets

In [38]:
# 2023 3 ayı validasyon, öncekilerde eğitim

# 2023'nin başına kadar (2022'nın sonuna kadar) train seti.
train = df.loc[(df["depremin_olus_zamani"] < "2023-01-01"), :]

# 2023'nin ilk 3'ayı validasyon seti.
val = df.loc[(df["depremin_olus_zamani"] >= "2023-01-01") & (df["depremin_olus_zamani"] < "2023-04-01"), :]

In [39]:
# train edilirken işimize yaramayacak olan kolonları çıkartalım. Buraya daha sonra enlem ve boylam eklenecek
cols = [col for col in df.columns if col not in ['deprem kodu',"no","yer","depremin_olus_zamani","temizlenmis_yer","xm"]]

In [40]:
# train => X_train, Y_train ....... val => X_val, Y_val
Y_train = train['xm']
X_train = train[cols]

Y_val = val['xm']
X_val = val[cols]

In [41]:
Y_train.shape, X_train.shape, Y_val.shape, X_val.shape

((18424,), (18424, 436), (1383,), (1383, 436))

## Optuna

Optuna'yı hiperparametre tuningi için kullanacağız. Zamandan tasarruf sağlaması için. GridSearchCV bizi çok yavaşlatıyor.

In [57]:
def objective(trial):
    # Define the hyperparameters to optimize
    max_iter = trial.suggest_int('max_iter', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 1.0)
    
    # Create and train the HistGradientBoostingRegressor
    model = HistGradientBoostingRegressor(
        max_iter=max_iter,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        learning_rate=learning_rate,
        random_state=42
    )
    
    # Train the model
    model.fit(X_train, Y_train)
    
    # Predict on the validation set
    y_pred_val = model.predict(X_val)
    
    # Calculate SMAPE on the validation set
    smape_val = smape(y_pred_val, Y_val)
    
    return smape_val

# Create an Optuna study
study = optuna.create_study(direction='minimize')  # Minimize SMAPE

# Optimize hyperparameters
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params

[I 2023-09-13 10:10:35,252] A new study created in memory with name: no-name-1d249c8f-858f-482e-90e5-f3fd995f0acb
[I 2023-09-13 10:10:36,329] Trial 0 finished with value: 4.452495705980679 and parameters: {'max_iter': 370, 'max_depth': 8, 'min_samples_leaf': 74, 'learning_rate': 0.9607184402241471}. Best is trial 0 with value: 4.452495705980679.
[I 2023-09-13 10:10:37,465] Trial 1 finished with value: 4.462329193066047 and parameters: {'max_iter': 163, 'max_depth': 22, 'min_samples_leaf': 50, 'learning_rate': 0.9079087147589494}. Best is trial 0 with value: 4.452495705980679.
[I 2023-09-13 10:10:39,488] Trial 2 finished with value: 4.214396012382098 and parameters: {'max_iter': 181, 'max_depth': 30, 'min_samples_leaf': 53, 'learning_rate': 0.19715240616558313}. Best is trial 2 with value: 4.214396012382098.
[I 2023-09-13 10:10:40,777] Trial 3 finished with value: 4.343207383786605 and parameters: {'max_iter': 265, 'max_depth': 31, 'min_samples_leaf': 7, 'learning_rate': 0.6534006736266

In [58]:
best_params

{'max_iter': 774,
 'max_depth': 2,
 'min_samples_leaf': 18,
 'learning_rate': 0.0018862398822892638}

## Model Developing

In [59]:
optimized_params = {
    'max_iter': 774,
    'max_depth': 2,
    'min_samples_leaf': 18,
    'learning_rate': 0.0018862398822892638
}

In [60]:
model = HistGradientBoostingRegressor(
    max_iter=optimized_params['max_iter'],
    max_depth=optimized_params['max_depth'],
    min_samples_leaf=optimized_params['min_samples_leaf'],
    learning_rate=optimized_params['learning_rate'],
    random_state=42  # You can set the random seed for reproducibility
)


In [61]:
model.fit(X_train, Y_train)

# Make predictions
y_pred = model.predict(X_val)

In [62]:
y_pred

array([1.58007766, 1.57776043, 1.57776043, ..., 1.58007766, 1.57803086,
       1.57803086])

In [63]:
hist_gradient_boosting_smape(y_pred,Y_val)

('SMAPE', 8.231142277443645, False)

## Creating PKL File

In [66]:
joblib.dump(model, "hgbr.pkl")

['hgbr.pkl']